## UseCase 1 - AI Travel Planner

**References**

1. [Integrating Foundation Model with Amazon Bedrock](https://aws.amazon.com/blogs/machine-learning/integrate-foundation-models-into-your-code-with-amazon-bedrock/)
2. [Working with Anthropic Claude Sonnet](https://aws.amazon.com/blogs/aws/anthropics-claude-3-5-sonnet-model-now-available-in-amazon-bedrock-the-most-intelligent-claude-model-yet/)

In [ ]:
import configparser
import json
import boto3
import logging
from botocore.exceptions import ClientError

### Initiate Services

In [ ]:
logging.basicConfig(
    format="%(levelname)s - %(asctime)s - %(message)s", level=logging.ERROR
)

logger = logging

### Reading Config File

In [ ]:
CONFIG_FILENAME=".config.ini"

config = configparser.ConfigParser()
config.read(CONFIG_FILENAME)

def get_value_by_section_and_key(section, key):
        """get_value_by_section_and_key"""
        return config.get(section, key)

def get_all_details_of_section(section) -> dict:
    """get_all_details_of_section"""
    return dict(config.items(section))


default_cfgs = get_all_details_of_section("DEFAULT")

### Create boto3 client - BEDROCK

In [ ]:
bedrock_client = boto3.client(
    'bedrock',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)


### Working With Model - Claude 3.5 Sonnet v2

In [ ]:
kwargs = {
    "modelId": "anthropic.claude-3-5-sonnet-20241022-v2:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1200,
        "messages": [
            {
                "role": "user",
                "content": [
                {
                    "type": "text",
                    "text": "I have can travel for 3 days in January 2025, and I prefer to go by bus from Bangalore."
                }
                ]
            }
        ],
        "system": "You are an expert in the domain of travel, and based on the month of travel, number of days, start point and mode of transport provided by the user, you can suggest them the best place to travel. Give a detailed itinerary for the number of days as well."
    }
}
# print(kwargs)

response = bedrock_runtime.invoke_model(
        modelId=kwargs["modelId"],
        body=json.dumps(kwargs["body"]).encode('utf-8')
    )
response_body = json.loads(response.get('body').read())
response_body